In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
zomato_real = pd.read_csv('/content/drive/MyDrive/zomato.csv',encoding='utf-8')

In [4]:
zomato_real.head(10)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
5,https://www.zomato.com/bangalore/timepass-dinn...,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,No,3.8/5,286,+91 9980040002\r\n+91 9980063005,Basavanagudi,Casual Dining,"Onion Rings, Pasta, Kadhai Paneer, Salads, Sal...",North Indian,600,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",[],Buffet,Banashankari
6,https://www.zomato.com/bangalore/rosewood-inte...,"19/1, New Timberyard Layout, Beside Satellite ...",Rosewood International Hotel - Bar & Restaurant,No,No,3.6/5,8,+91 9731716688\r\n080 26740366,Mysore Road,Casual Dining,NaN,"North Indian, South Indian, Andhra, Chinese",800,"[('Rated 5.0', 'RATED\n Awesome food ??Great ...",[],Buffet,Banashankari
7,https://www.zomato.com/bangalore/onesta-banash...,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,Yes,Yes,4.6/5,2556,080 48653961\r\n080 48655715,Banashankari,"Casual Dining, Cafe","Farmhouse Pizza, Chocolate Banana, Virgin Moji...","Pizza, Cafe, Italian",600,"[('Rated 5.0', 'RATED\n I personally really l...",[],Cafes,Banashankari
8,https://www.zomato.com/bangalore/penthouse-caf...,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,Yes,No,4.0/5,324,+91 8884135549\r\n+91 9449449316,Banashankari,Cafe,"Pizza, Mocktails, Coffee, Nachos, Salad, Pasta...","Cafe, Italian, Continental",700,"[('Rated 3.0', ""RATED\n I had been to this pl...",[],Cafes,Banashankari
9,https://www.zomato.com/bangalore/smacznego-ban...,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,Yes,No,4.2/5,504,+91 9945230807\r\n+91 9743804471,Banashankari,Cafe,"Waffles, Pasta, Coleslaw Sandwich, Choco Waffl...","Cafe, Mexican, Italian, Momos, Beverages",550,"[('Rated 4.0', ""RATED\n Easy to locate\nVFM 3...",[],Cafes,Banashankari


In [5]:
zomato_real.shape

(51717, 17)

In [6]:
zomato=zomato_real.drop(['url','dish_liked','phone','listed_in(type)','listed_in(city)','phone'],axis='columns')

In [7]:
zomato = zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type',
                                  'listed_in(city)':'city'}) #it renames the columns with cost,type and city
zomato.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item'],
      dtype='object')

In [8]:
print(zomato['location'].unique())

['Banashankari' 'Basavanagudi' 'Mysore Road' 'Jayanagar'
 'Kumaraswamy Layout' 'Rajarajeshwari Nagar' 'Vijay Nagar' 'Uttarahalli'
 'JP Nagar' 'South Bangalore' 'City Market' 'Nagarbhavi'
 'Bannerghatta Road' 'BTM' 'Kanakapura Road' 'Bommanahalli' nan
 'CV Raman Nagar' 'Electronic City' 'HSR' 'Marathahalli' 'Sarjapur Road'
 'Wilson Garden' 'Shanti Nagar' 'Koramangala 5th Block'
 'Koramangala 8th Block' 'Richmond Road' 'Koramangala 7th Block'
 'Jalahalli' 'Koramangala 4th Block' 'Bellandur' 'Whitefield'
 'East Bangalore' 'Old Airport Road' 'Indiranagar' 'Koramangala 1st Block'
 'Frazer Town' 'RT Nagar' 'MG Road' 'Brigade Road' 'Lavelle Road'
 'Church Street' 'Ulsoor' 'Residency Road' 'Shivajinagar' 'Infantry Road'
 'St. Marks Road' 'Cunningham Road' 'Race Course Road' 'Commercial Street'
 'Vasanth Nagar' 'HBR Layout' 'Domlur' 'Ejipura' 'Jeevan Bhima Nagar'
 'Old Madras Road' 'Malleshwaram' 'Seshadripuram' 'Kammanahalli'
 'Koramangala 6th Block' 'Majestic' 'Langford Town' 'Central Bangalo

In [9]:
import ast

In [10]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['cuisines'])
    return L

In [11]:
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[]
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[]
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[]
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[]
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[]


In [12]:
zomato.dropna(inplace=True)

In [13]:
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[]
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[]
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[]
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[]
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[]


In [14]:
zomato['cuisines'] = zomato['cuisines'].str.replace(r'\s+', '', regex=True)

In [15]:
zomato['location'] = zomato['location'].str.replace(r'\s+', '', regex=True)

In [16]:
zomato.head(11)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"NorthIndian,Mughlai,Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[]
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese,NorthIndian,Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[]
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe,Mexican,Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[]
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"SouthIndian,NorthIndian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[]
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"NorthIndian,Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[]
5,"37, 5-1, 4th Floor, Bosco Court, Gandhi Bazaar...",Timepass Dinner,Yes,No,3.8/5,286,Basavanagudi,Casual Dining,NorthIndian,600,"[('Rated 3.0', 'RATED\n Food 3/5\nAmbience 3/...",[]
6,"19/1, New Timberyard Layout, Beside Satellite ...",Rosewood International Hotel - Bar & Restaurant,No,No,3.6/5,8,MysoreRoad,Casual Dining,"NorthIndian,SouthIndian,Andhra,Chinese",800,"[('Rated 5.0', 'RATED\n Awesome food ??Great ...",[]
7,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,Yes,Yes,4.6/5,2556,Banashankari,"Casual Dining, Cafe","Pizza,Cafe,Italian",600,"[('Rated 5.0', 'RATED\n I personally really l...",[]
8,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,Yes,No,4.0/5,324,Banashankari,Cafe,"Cafe,Italian,Continental",700,"[('Rated 3.0', ""RATED\n I had been to this pl...",[]
9,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,Yes,No,4.2/5,504,Banashankari,Cafe,"Cafe,Mexican,Italian,Momos,Beverages",550,"[('Rated 4.0', ""RATED\n Easy to locate\nVFM 3...",[]


In [17]:
zomato["cuisines"] = zomato["cuisines"].str.lower()

In [18]:
zomato["location"] = zomato["location"].str.lower()

In [19]:
print(zomato['cuisines'])

0                   northindian,mughlai,chinese
1                      chinese,northindian,thai
2                          cafe,mexican,italian
3                       southindian,northindian
4                        northindian,rajasthani
                          ...                  
51709                   northindian,continental
51711    andhra,southindian,chinese,northindian
51712                               continental
51715                                fingerfood
51716        fingerfood,northindian,continental
Name: cuisines, Length: 43533, dtype: object


In [20]:
zomato.head(4)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,banashankari,Casual Dining,"northindian,mughlai,chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[]
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,banashankari,Casual Dining,"chinese,northindian,thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[]
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,banashankari,"Cafe, Casual Dining","cafe,mexican,italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[]
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,banashankari,Quick Bites,"southindian,northindian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[]


In [21]:
def convert(text):
        # Split the text using commas as the delimiter
        values = text.split(',')

        # Return the resulting list
        return values


for i, value in zomato['cuisines'].iteritems():
    zomato['cuisines'][i]=convert(zomato['cuisines'][i])

In [22]:
def convert(text):
        # Split the text using commas as the delimiter
        values = text.split(',')

        # Return the resulting list
        return values


for i, value in zomato['location'].iteritems():
    zomato['location'][i]=convert(zomato['location'][i])

In [23]:
zomato

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,[banashankari],Casual Dining,"[northindian, mughlai, chinese]",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[]
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,[banashankari],Casual Dining,"[chinese, northindian, thai]",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[]
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,[banashankari],"Cafe, Casual Dining","[cafe, mexican, italian]",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[]
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,[banashankari],Quick Bites,"[southindian, northindian]",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[]
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,[basavanagudi],Casual Dining,"[northindian, rajasthani]",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
51709,"136, SAP Labs India, KIADB Export Promotion In...",The Farm House Bar n Grill,No,No,3.7 /5,34,[whitefield],"Casual Dining, Bar","[northindian, continental]",800,"[('Rated 4.0', 'RATED\n Ambience- Big and spa...",[]
51711,"139/C1, Next To GR Tech Park, Pattandur Agraha...",Bhagini,No,No,2.5 /5,81,[whitefield],"Casual Dining, Bar","[andhra, southindian, chinese, northindian]",800,"[('Rated 4.0', 'RATED\n A fine place to chill...",[]
51712,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,[whitefield],Bar,[continental],"1,500","[('Rated 5.0', ""RATED\n Food and service are ...",[]
51715,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,"[itplmainroad, whitefield]",Bar,[fingerfood],"2,500","[('Rated 4.0', 'RATED\n Nice and friendly pla...",[]


In [24]:
#movies['overview'] = movies['overview'].apply(lambda x:x.split())
zomato.sample(5)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item
39326,"Hotel Southern Star, 40/2, Roof Top, Lavelle R...",Tree Tops Bar & Kitchen,No,Yes,4.4 /5,130,[lavelleroad],"Casual Dining, Bar","[northindian, chinese, continental, italian]","1,000","[('Rated 5.0', ""RATED\n Love how lively this ...",[]
35807,"31/41, Near Post Office, 10th Cross, H Siddaih...",Hotel Ruchi Priya,Yes,No,3.6 /5,22,[wilsongarden],Quick Bites,[northindian],350,"[('Rated 3.0', ""RATED\n Do not go to this pla...",[]
23924,"5th Main, 5th Cross, 5th Block, 1st Stage, HBR...",Paul's Dum Biryani,Yes,No,4.0/5,148,[hbrlayout],Delivery,"[biryani, hyderabadi]",400,"[('Rated 4.0', 'RATED\n Yummy Biryani, with a...",[]
42242,"9/3, Prestige Pearl House, Museum Road, Church...",Little Green Cafe,No,No,4.3 /5,795,[churchstreet],"Cafe, Casual Dining","[cafe, healthyfood, salad, continental, bevera...",900,"[('Rated 4.0', ""RATED\n As the name suggests,...",[]
29853,"8, 3rd A Cross, 31st Main, 2nd Stage, BTM, Ban...",Speedbreaker,Yes,No,3.4 /5,15,[btm],Quick Bites,[northindian],300,"[('Rated 4.0', 'RATED\n Its a good place to h...","['Chole with 2 Bhatura', 'Chicken Fry', '2 Alo..."


In [25]:
zomato['harsh'] = zomato['location'] + zomato['cuisines']

In [26]:
new = zomato.drop(columns=['location','cuisines','menu_item'])
#new.head()

In [27]:
new['rate'].dtype

dtype('O')

In [28]:
new['harsh'] = new['harsh'].apply(lambda x: " ".join(x))
new.head()

,address,name,online_order,book_table,rate,votes,rest_type,cost,reviews_list,harsh
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Casual Dining,800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",banashankari northindian mughlai chinese
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Casual Dining,800,"[('Rated 4.0', 'RATED\n Had been here for din...",banashankari chinese northindian thai
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,"Cafe, Casual Dining",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",banashankari cafe mexican italian
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Quick Bites,300,"[('Rated 4.0', ""RATED\n Great food and proper...",banashankari southindian northindian
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Casual Dining,600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",basavanagudi northindian rajasthani


In [29]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(new['harsh'])





In [30]:
vectorizer = TfidfVectorizer()
feature_matrix = vectorizer.fit_transform(new['harsh'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(feature_matrix, dense_output=False)


In [31]:
# Function to get restaurant recommendations based on cosine similarity
def get_recommendations(user_cuisine, user_location):
    user_input = f'{user_cuisine} {user_location}'

    # Vectorize user input
    user_vector = vectorizer.transform([user_input])

    # Calculate cosine similarity between user input and existing restaurants
    similarities = cosine_similarity(user_vector, feature_matrix).flatten()

    # Get indices of restaurants sorted by similarity (in descending order)
    similar_restaurants_indices = similarities.argsort()[::-1]

    # Get restaurant names based on similarity
    recommended_restaurants = new.iloc[similar_restaurants_indices]


    return recommended_restaurants

# Example usage
user_cuisine = 'southindian'
user_location = 'banashankari'

recommendations = get_recommendations(user_cuisine, user_location)

print("Top 5 restaurent based on rating in Bengaluru\n")
recommendations.head(5)

Top 5 restaurent based on rating in Bengaluru



,address,name,online_order,book_table,rate,votes,rest_type,cost,reviews_list,harsh
479,"406, 2nd Cross, Vidyapeeta Layout, Banashankri...",Kababs N Biryani,Yes,No,3.4/5,22,Quick Bites,200,"[('Rated 1.0', ""RATED\n I orderd Chiken Biriy...",banashankari southindian
39,"556/A, Water Tank Road, 3rd Stage, Banashankar...",Maruthi Davangere Benne Dosa,Yes,No,4.0/5,17,Quick Bites,150,"[('Rated 5.0', ""RATED\n This restaurant serve...",banashankari southindian
2616,"17, Ground Floor, Near Big Bazar, 100 Feet Rin...",Gama Gama,Yes,No,3.9/5,264,Quick Bites,200,"[('Rated 4.0', 'RATED\n Sunday Noon and my fr...",banashankari southindian
2614,"162 , Kathreguppe Cross Road, 3rd Stage, Banas...",Janahaar,Yes,No,3.8/5,70,Quick Bites,100,"[('Rated 5.0', 'RATED\n GÃ\x83Ã\x83Ã\x82Ã...",banashankari southindian
2598,"855/A, 5th A Cross,10th Main Road, Srinivasana...",Bhattara Bhojana,Yes,No,3.6/5,66,Quick Bites,150,"[('Rated 1.0', 'RATED\n I packed a full meal ...",banashankari southindian
